In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
br_stocks = pd.read_csv("acoes_brasil.csv", sep=";", decimal=",", thousands='.').fillna(0)
print(br_stocks.columns)
# print(br_stocks.dtypes)
br_stocks.head()

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')


,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,...,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
0,AALR3,10.41,0.00,-4.30,1.42,0.49,30.29,-1.09,-25.30,-99.75,...,0.34,0.65,0.45,0.13,0.00,1.898619e+06,7.32,-2.42,-0.01,1.231428e+09
1,ABCB4,19.86,6.07,5.71,0.83,0.08,31.62,25.25,19.35,4.38,...,0.10,0.90,0.07,18.70,12.95,1.226095e+07,23.98,3.48,0.39,4.490150e+09
2,ABEV3,12.98,5.86,14.34,2.41,1.54,49.95,21.50,17.26,11.52,...,0.64,0.35,0.62,10.72,14.46,3.329247e+08,5.40,0.91,2.64,2.047998e+11
3,ADHM3,1.56,0.00,-4.09,-1.29,105.12,100.00,-13548.48,-18860.61,-5.69,...,-81.72,82.72,0.14,0.00,0.00,0.000000e+00,-1.21,-0.38,-0.58,2.543977e+07
4,AERI3,0.81,0.00,-5.82,0.74,0.20,13.03,9.37,-3.59,2.23,...,0.27,0.73,0.93,0.00,0.00,6.355384e+06,1.10,-0.14,0.00,6.206329e+08


In [15]:
good_price_equity = (lambda df : df[ df["P/L"] > 3 ].sort_values(by="P/L").head(20))
good_price_equity(br_stocks)

,TICKER,PRECO,DY,P/L,P/VP,P/ATIVOS,MARGEM BRUTA,MARGEM EBIT,MARG. LIQUIDA,P/EBIT,...,PATRIMONIO / ATIVOS,PASSIVOS / ATIVOS,GIRO ATIVOS,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,LIQUIDEZ MEDIA DIARIA,VPA,LPA,PEG Ratio,VALOR DE MERCADO
168,CORR3,2.71,2762.62,0.01,0.01,0.00,100.00,2501.40,2192.46,0.01,...,0.32,0.68,0.02,-16.36,0.00,0.00,263.25,339.01,0.00,1.278895e+06
312,HETA4,6.37,0.00,0.02,-0.01,0.24,100.00,66.20,6168.36,1.47,...,-25.49,26.49,0.24,3.27,0.00,1501.00,-689.01,403.24,0.00,1.859317e+07
388,MMAQ4,49.73,503.40,0.06,0.01,0.00,9.67,6.17,4.08,0.04,...,0.47,0.53,1.82,25.76,29.54,0.00,5503.84,873.91,-0.03,1.115195e+06
87,BPHA3,0.62,0.00,0.06,-0.24,0.99,98.02,1029.99,13896.67,0.80,...,-4.15,5.15,0.12,-68.85,0.00,0.00,-2.58,10.40,0.00,7.011030e+07
33,ATMP3,2.85,0.00,0.07,-0.22,0.05,-22.72,-46.86,258.67,-0.39,...,-0.21,1.21,0.25,-16.95,0.00,197344.76,-13.18,40.30,0.00,7.612451e+07
390,MNDL3,55.96,0.00,0.15,0.82,0.11,37.96,12.74,112.86,1.35,...,0.13,0.87,0.63,11.73,0.00,42635.70,68.56,368.06,0.00,1.387953e+08
526,SLED4,1.60,0.00,0.22,-0.04,0.24,47.85,73.94,92.71,0.28,...,-5.92,6.92,1.15,-45.03,0.00,283562.78,-39.57,7.15,-0.01,1.649769e+07
169,CORR4,82.00,91.30,0.24,0.31,0.10,100.00,2501.40,2192.46,0.21,...,0.32,0.68,0.02,-16.36,0.00,0.00,263.25,339.01,0.00,1.278895e+06
311,HETA3,96.00,0.00,0.24,-0.14,3.55,100.00,66.20,6168.36,22.18,...,-25.49,26.49,0.24,3.27,0.00,0.00,-689.01,403.24,0.00,1.859317e+07
53,BDLL4,9.20,0.00,0.27,-0.10,0.02,-48.56,-81.27,144.73,-0.47,...,-0.23,1.23,0.06,-27.58,0.00,4695.79,-88.93,34.54,0.00,1.569151e+07
